In [16]:
import math
import statistics
from sklearn import linear_model, datasets

def grayscale(img):
    """Applies the Grayscale transform"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def v_point(right, left):
    """
    Calculates the vanishing point.
    
    Given the right and left lane lines, it first calculate the 
    respective averages for the lane lines, and then uses the 
    line equations to find the vanishing point.
    """
    right_avg=average(right)
    left_avg=average(left)
    
    right_b = right_avg[5]
    right_slope = right_avg[4]
    left_b = left_avg[5]
    left_slope = left_avg[4]
    
    x = int((right_b - left_b) / (left_slope - right_slope))
    y = int(x * left_slope + left_b)
    
    return (x,y)

ctr=0
right_b=0
right_slope=0
left_b=0
left_slope=0
def draw_using_moving_avg(right, left, img):
    """
    Draws lines, keeping a global average.
    
    Very basic implementation, keeps a global average of the line's historical values.
    This implementation is very basic and in reality would cause a lot of issues, on 
    mainly is the fact that this kind of implementation would make the detected lines
    to react very slowly to changing conditions.
    """
    
    global right_b
    global right_slope
    global left_b
    global left_slope
    global ctr
    
    right_avg = average(right)
    left_avg = average(left)
    
    right_b = (right_b * ctr + right_avg[5]) / (ctr + 1)
    right_slope = (right_slope * ctr + right_avg[4]) / (ctr + 1)

    left_b = (left_b * ctr + left_avg[5]) / (ctr + 1)
    left_slope = (left_slope * ctr + left_avg[4]) / (ctr + 1)
    ctr += 1
    
    y1 = int(img.shape[0])
    x1 = int((y1-right_b)/right_slope)

    y2 = int(.61 * img.shape[0])
    x2 = int((y2-right_b)/right_slope)
    cv2.line(img, (x1,y1), (x2,y2), [255,0,0], 15)

    y1 = int(img.shape[0])
    x1 = int((y1-left_b)/left_slope)

    y2 = int(.61 * img.shape[0])
    x2 = int((y2-left_b)/left_slope)
    cv2.line(img, (x1,y1), (x2,y2), [255,0,0], 15)


def average(lines):
    """
    Helper function to find the average of values.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    
    lines = [item for sublist in lines for item in sublist]
    N = float(len(lines))
    return tuple(sum(col)/N for col in zip(*lines))    

def filter_vpoint(v_point, right, left):
    """
    Filters right and left lane lines based on the vanishing point    
    """
    right_result = []
    left_result = []

    for line in right:
        for x1,y1,x2,y2,m,b in line:
            if (x1 <=  v_point[0] or x2 <= v_point[0]):
                continue
            right_result.append([[x1,y1,x2,y2,m,b]])

    for line in left:
        for x1,y1,x2,y2,m,b in line:
            if (x1 >=  v_point[0] or x2 >= v_point[0]):
                continue
            left_result.append([[x1,y1,x2,y2,m,b]])
        
    return (right_result, left_result)        

def filter_far_lines(lines):
    """
    Filters out lines that are too far.    
    """
    avg = average(lines)
    avg_m = avg[4]
    avg_b = avg[5]
    
    result=[]
    
    
    for line in lines:
        for x1,y1,x2,y2,m,b in line:
            d = abs(b-avg_b) / math.sqrt(pow(m,2)+1)
            if (d <= 250):
                result.append([[x1,y1,x2,y2,m,b]])
            #else:
                #print("ignoring %d", d)

    return result                
                
def draw_lines_simple(img, lines, color=[255, 0, 0], thickness=10):
    """
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    """
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, 2)

def draw_extrapolated_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    Does the following steps
    1) separates out lines into left/right based on slope
    2) removes lines that are too flat or too vertical
    3) finds the vanishing point
    4) filter out noisy lines, using the vanishing point
    5) calculates avg slopes for right/left
    6) filter out noise lines, that are too far away from the average
    7) draws lines based on avg slopes and the vanishing point
    8) keeps a running avg of the right/left slopes 
    """
    right=[]
    left=[]
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            m = ((y2-y1)/(x2-x1))
            if (m == float("-inf") or m == float("inf")):
                continue
            if ((m < 0.55 and m > 0) or (m > -0.55 and m < 0)):
                continue               

            b = y2 - x2 * m
            if (m < 0):
                left.append([[x1,y1,x2,y2,m,b]])
            if (m > 0):
                right.append([[x1,y1,x2,y2,m,b]])
          
    v_p = v_point(right, left)

    right,left = filter_vpoint(v_p, right, left)
    
    avg = average(right)
    avg_m = avg[4]
    avg_b = avg[5]
    
    right = filter_far_lines(right)
    left = filter_far_lines(left)
    draw_using_moving_avg(right, left, img)
    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, simple=False):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    if (simple):
        draw_lines_simple(line_img, lines)
    else:            
        draw_extrapolated_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [25]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os

import cv2
%matplotlib inline

def pipeline(image, simple=False):
    """
    1) Applies the Grayscale transform
    2) Applies a Gaussian Noise kernel
    3) Applies the Canny transform
    4) Draws hough lines
    5) then draws the lines 

    In simple mode, it will draw the hough lines, otherwise, it will do the following:

    1) separates out lines into left/right based on slope
    2) removes lines that are too flat or too vertical
    3) finds the vanishing point
    4) filter out noisy lines, using the vanishing point
    5) calculates avg slopes for right/left
    6) filter out noise lines, that are too far away from the average
    7) draws lines based on avg slopes and the vanishing point
    8) keeps a running avg of the right/left slopes 

    """
    
    gray = grayscale(image)
    imshape = image.shape

    kernel_size = 3
    blur_gray = gaussian_blur(gray, kernel_size)

    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    vertices = np.array([[(imshape[1]/16, imshape[0]), (7*imshape[1]/16, 61*imshape[0]/100),(9*imshape[1]/16, 61*imshape[0]/100), (15*imshape[1]/16, imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)

    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 1     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 2 #minimum number of pixels making up a line
    max_line_gap = 2    # maximum gap in pixels between connectable line segments
    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap, simple)
        
    #color_edges = np.dstack((edges, edges, edges)) 

    lines_edges = weighted_img(line_image, image)
    return lines_edges

test_image_file_names = os.listdir("test_images/")
for image_file_name in test_image_file_names:
    print(image_file_name)
    # Read in and grayscale the image
    image = mpimg.imread("test_images/" + image_file_name)    
    result = pipeline(image, True)
    mpimg.imsave("test_image_hough_lines/" + image_file_name, result)

solidWhiteCurve.jpg
solidWhiteRight.jpg
solidYellowCurve.jpg
solidYellowCurve2.jpg
solidYellowLeft.jpg
whiteCarLaneSwitch.jpg


In [34]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    result = pipeline(image)
    # you should return the final output (image where lines are drawn on lanes)
    return result

white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:03<00:00, 58.41it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 3.32 s, sys: 250 ms, total: 3.57 s
Wall time: 4.22 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [36]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:12<00:00, 53.38it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 10.3 s, sys: 920 ms, total: 11.3 s
Wall time: 13.2 s


In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [37]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:08<00:00, 28.90it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 6.47 s, sys: 330 ms, total: 6.8 s
Wall time: 9.57 s


In [29]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))